# Playground Notebook
## To try, validate and debug the code into the dashboard

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import os

## Stocks

In [2]:
# Stock Names
stock_names = {'GS':'Goldman Sachs Group Inc',
               'LMT':'Lockheed Martin Corporation',
               'TSLA':'TESLA',
               'MSFT':'Microsoft Corporation',
               'AAPL':'Apple Inc.',
               'MCD':'McDonalds Corporation',
               'NKE':'Nike Inc',
               'PFE':'Pfizer Inc.',
               'FB':'Facebook, Inc.',
               'GOOGL':'Alphabet Inc.'
              }

In [3]:
# Name of timestamp to parse on the raw data file
stamp_name = '4. timestamp'

In [4]:
# Column names on the raw data file
header_names = {'1. symbol':'sym',
                '2. price':'price_str', 
                '3. volume':'vol'}

In [5]:
# Directories and Files
# Windows
if os.name == 'nt':
    home_dir = r'//userhome/users$/ksagilop/home/ZHAW/MAIN/04_Big_Data/'
    stock_dir = r'spark-stock-market-streaming/collected_data/'
    tweet_dir = r'spark-stock-market-streaming/collected_tweets_csv_raw/'
# Linux
if os.name == 'posix':
    home_dir = os.path.expanduser(r'~/Documents/ZHAW/MAIN/04_Big_Data/30_Project/')
    stock_dir = r'spark-stock-market-streaming/collected_data/'
    tweet_dir = r'spark-stock-market-streaming/collected_tweets_csv_raw/'

### Build Dataframe from json files

In [6]:
def buildDF(base_dir, data_dir, json_col):
    '''Construct a big dataframe by aggregating the individual json files
    located at the proper data directory
    Args:
        base_dir(str), the home or base directory
        data_dir(str), the directory containing the data
        json_col(str), which column to normalize from the json file
    return:
        df(dataframe), full dataframe iaw json structure'''
    folder = os.path.join(base_dir + data_dir)
    files = os.listdir(folder)
    count_files = 0
    for file in files:
        file_path = os.path.join(folder + file)
        with open(file_path) as data_file:
            data = json.load(data_file)
            if count_files == 0:
                df = json_normalize(data, json_col)
                print('---- Base Dataframe ----')
                print('Lenght of base dataframe is: ', len(df))
                print(file_path)
                print(df.head())
                count_files += 1
            else:
                try:
                    df_temp = json_normalize(data, json_col)
                    df = df.append(df_temp, ignore_index=True)
                    count_files += 1
                except:
                    print(file_path, 'normalize failed')
                    continue
    print('------------------------')
    print('Total files read: ' + str(count_files))
    print('---- %d Dataframes appended ----' %count_files)
    print('Total lenght of dataframe is: ', len(df))
    return df

In [7]:
dfs = buildDF(home_dir, stock_dir, 'Stock Quotes')

---- Base Dataframe ----
Lenght of base dataframe is:  10
/home/hase/Documents/ZHAW/MAIN/04_Big_Data/30_Project/spark-stock-market-streaming/collected_data/20180614-152855.json
  1. symbol  2. price 3. volume         4. timestamp
0      TSLA  344.7800        --  2018-06-13 15:59:58
1      AAPL  190.6650        --  2018-06-13 15:59:57
2      MSFT  100.8900        --  2018-06-13 15:59:58
3       MCD  166.6300        --  2018-06-13 15:59:59
4       NKE   74.1200        --  2018-06-13 15:59:56
/home/hase/Documents/ZHAW/MAIN/04_Big_Data/30_Project/spark-stock-market-streaming/collected_data/20180614-180447.json normalize failed
/home/hase/Documents/ZHAW/MAIN/04_Big_Data/30_Project/spark-stock-market-streaming/collected_data/20180614-175150.json normalize failed
/home/hase/Documents/ZHAW/MAIN/04_Big_Data/30_Project/spark-stock-market-streaming/collected_data/20180614-173053.json normalize failed
------------------------
Total files read: 3149
---- 3149 Dataframes appended ----
Total lenght o

### Data Pre-processing

In [8]:
def transDF(df, stamp, header, names):
    '''Pre-processing of the data by doing some transformations
    and aggregations to the dataframe created previously
    Args:
        df(dataframe): input dataframe
        stamp(str): name column to parse for datetime
        header(dict): a dictionary with {'original col name':'new col name'}
        names(dict): a dictionary with the mapping {'stock abrev.':'stock full name'}
    return:
        df(dataframe): transformed dataframe'''
    print('---- Input Dataframe ----')
    print(df.head())
    # Parse date and time
    print('------------------------')
    print('Parsing datetimes...')
    df['stamp'] = pd.to_datetime(df[stamp])
    df['date'] = df['stamp'].dt.date
    #df['year'] = df['stamp'].dt.year
    #df['month'] = df['stamp'].dt.strftime('%b')
    #df['day'] = df['stamp'].dt.day
    df['time'] = df['stamp'].dt.time
    #print('Dropping auxiliary columns...')
    df.drop([stamp], axis=1, inplace=True) # Not needed as 'stamp' is used for x-axis
    # Set dataframe index
    df.set_index(['date', 'time'], inplace=True)
    #df.set_index(['year', 'month', 'day', 'time'], inplace=True)
    print('Multi-index set from: ', stamp)
    print('Renaming columns...')
    df.rename(columns=header, inplace=True)
    #print(df.head())
    print('Change dtype for price...')
    df['price'] = pd.to_numeric(df['price_str'])
    df.drop(['price_str'], axis=1, inplace=True)
    print('Drop duplicates...')
    original_len = len(df)
    df.drop_duplicates(inplace=True)
    drop_len = len(df)
    print('Dataframe reduced from %d to %d rows' %(original_len, drop_len))
    df.sort_index(inplace=True)
    print('Renaming rows...')
    for key, value in names.items():
        mask = df.sym == key
        df.loc[mask, 'sym'] = value + ', ' + '(' + key + ')'
    print('---- Modified Dataframe ----')
    print(df.head())
    print(df.dtypes)
    return df

In [9]:
dfss = transDF(dfs, stamp_name, header_names, stock_names)

---- Input Dataframe ----
  1. symbol  2. price 3. volume         4. timestamp
0      TSLA  344.7800        --  2018-06-13 15:59:58
1      AAPL  190.6650        --  2018-06-13 15:59:57
2      MSFT  100.8900        --  2018-06-13 15:59:58
3       MCD  166.6300        --  2018-06-13 15:59:59
4       NKE   74.1200        --  2018-06-13 15:59:56
------------------------
Parsing datetimes...
Multi-index set from:  4. timestamp
Renaming columns...
Change dtype for price...
Drop duplicates...
Dataframe reduced from 31490 to 22345 rows
Renaming rows...
---- Modified Dataframe ----
                                                    sym vol  \
date       time                                               
2018-06-05 15:59:46       Goldman Sachs Group Inc, (GS)  --   
           15:59:53  Lockheed Martin Corporation, (LMT)  --   
           15:59:55                       TESLA, (TSLA)  --   
           15:59:55       Microsoft Corporation, (MSFT)  --   
           15:59:57                  Apple

In [10]:
dfss.head()

sym vol  \
date       time                                               
2018-06-05 15:59:46       Goldman Sachs Group Inc, (GS)  --   
           15:59:53  Lockheed Martin Corporation, (LMT)  --   
           15:59:55                       TESLA, (TSLA)  --   
           15:59:55       Microsoft Corporation, (MSFT)  --   
           15:59:57                  Apple Inc., (AAPL)  --   

                                  stamp   price  
date       time                                  
2018-06-05 15:59:46 2018-06-05 15:59:46  228.30  
           15:59:53 2018-06-05 15:59:53  320.13  
           15:59:55 2018-06-05 15:59:55  290.94  
           15:59:55 2018-06-05 15:59:55  102.21  
           15:59:57 2018-06-05 15:59:57  193.36

In [ ]:
dfss.price.mean()

In [ ]:
s_hr_min = dfss.index.get_level_values('time').min()
s_hr_max = dfss.index.get_level_values('time').max()

In [ ]:
s_hr_min, s_hr_max

## Tweets

In [11]:
hashtag_map = {'nike':'Nike Inc, (NKE)', 
               'facebook':'Facebook, Inc., (FB)', 
               'apple':'Apple Inc., (AAPL)', 
               'microsoft':'Microsoft Corporation, (MSFT)', 
               'mcdonalds':'McDonalds Corporation, (MCD)', 
               'tesla':'TESLA, (TSLA)',
               'n/a':'NA', 
               'goldmansachs':'Goldman Sachs Group Inc, (GS)', 
               'alphabet':'Alphabet Inc., (GOOGL)', 
               'pfizer':'Pfizer Inc., (PFE)'}

In [12]:
def buildDFbis(base_dir, data_dir):
    '''Construct a big dataframe by aggregating the individual csv files
    located at the proper data directory
    Args:
        base_dir(str), the home or base directory
        data_dir(str), the directory containing the data
    return:
        df(dataframe), full dataframe iaw csv structure'''
    folder = os.path.join(base_dir + data_dir)
    files = os.listdir(folder)
    file_path = os.path.join(folder + files[0])
    # Dataframe
    df = pd.read_csv(file_path,
                     sep=';',
                     parse_dates={'stamp':['timestamp']},
                     keep_date_col=False)
    print('Dataframe created from: ', file_path)
    print('with length: ', len(df))
    print('--- Dataframe processing ---')
    print('Creating date and time columns from timestamp')
    df['date'] = df['stamp'].dt.date
    df['time'] = df['stamp'].dt.time
    print('Setting date and time as Index...')
    df.set_index(['date', 'time'], inplace=True)
    return df

In [13]:
dft = buildDFbis(home_dir, tweet_dir)

Dataframe created from:  /home/hase/Documents/ZHAW/MAIN/04_Big_Data/30_Project/spark-stock-market-streaming/collected_tweets_csv_raw/tweets.csv
with length:  171273
--- Dataframe processing ---
Creating date and time columns from timestamp
Setting date and time as Index...


In [14]:
dft.head()

stamp   hashtag  cnt  followers_count  \
date       time                                                           
2018-06-09 03:10:23 2018-06-09 03:10:23     apple    1              287   
           03:10:28 2018-06-09 03:10:28     apple    1              535   
           03:10:30 2018-06-09 03:10:30      nike    1               66   
           03:10:32 2018-06-09 03:10:32     apple    1            39384   
           03:10:33 2018-06-09 03:10:33  facebook    1            11822   

                     log_followers_count  
date       time                           
2018-06-09 03:10:23             5.659482  
           03:10:28             6.282267  
           03:10:30             4.189655  
           03:10:32            10.581115  
           03:10:33             9.377717

In [ ]:
# Slice tweet dataframe to an equal stock timestamp
stock_hr_min = dfss.index.get_level_values('time').min()
stock_hr_max = dfss.index.get_level_values('time').max()
tweet_hr_min = dft.index.get_level_values('time').min()
tweet_hr_max = dft.index.get_level_values('time').max()
mask_hr_min = dft.index.get_level_values('time') > stock_hr_min
dft = dft.loc[mask_hr_min]
mask_hr_max = dft.index.get_level_values('time') < stock_hr_max
dft = dft.loc[mask_hr_max]

In [ ]:
dft.head()

In [ ]:
dft['stamp_round'] = dft.stamp.dt.round('1Min')

In [ ]:
c = dft.groupby(['stamp_round'])['log_followers_count'].sum()

In [ ]:
c.head()

In [ ]:
c.index

In [ ]:
c

In [ ]:
dft.head()

In [ ]:
dft.groupby(['stamp', 'time_min'])['log_followers_count'].sum()

In [16]:
Y_tweet = dft.groupby(['stamp'])['log_followers_count'].sum()

In [18]:
Y_tweet.max()

432.08252055888306

In [26]:
Y_tweet.idxmax()

Timestamp('2018-06-13 01:07:41')

In [27]:
dfss.head()

sym vol  \
date       time                                               
2018-06-05 15:59:46       Goldman Sachs Group Inc, (GS)  --   
           15:59:53  Lockheed Martin Corporation, (LMT)  --   
           15:59:55                       TESLA, (TSLA)  --   
           15:59:55       Microsoft Corporation, (MSFT)  --   
           15:59:57                  Apple Inc., (AAPL)  --   

                                  stamp   price  
date       time                                  
2018-06-05 15:59:46 2018-06-05 15:59:46  228.30  
           15:59:53 2018-06-05 15:59:53  320.13  
           15:59:55 2018-06-05 15:59:55  290.94  
           15:59:55 2018-06-05 15:59:55  102.21  
           15:59:57 2018-06-05 15:59:57  193.36

In [28]:
dfss.price.max()

1165.7

In [33]:
dfss.stamp[dfss.price == dfss.price.max()]

date        time    
2018-06-14  13:31:36   2018-06-14 13:31:36
            13:32:31   2018-06-14 13:32:31
Name: stamp, dtype: datetime64[ns]

In [34]:
dfss[dfss['stamp'] == '2018-06-14 13:31:36']

sym       vol  \
date       time                                                
2018-06-14 13:31:36                  TESLA, (TSLA)   7337523   
           13:31:36             Apple Inc., (AAPL)  11768908   
           13:31:36  Microsoft Corporation, (MSFT)  12972808   
           13:31:36         Alphabet Inc., (GOOGL)   1000038   

                                  stamp     price  
date       time                                    
2018-06-14 13:31:36 2018-06-14 13:31:36   354.565  
           13:31:36 2018-06-14 13:31:36   191.090  
           13:31:36 2018-06-14 13:31:36   101.620  
           13:31:36 2018-06-14 13:31:36  1165.700

In [35]:
dfss[dfss['stamp'] == '2018-06-14 13:32:31']

,,sym,vol,stamp,price
date,time,,,,
2018-06-14,13:32:31,"Alphabet Inc., (GOOGL)",1001575,2018-06-14 13:32:31,1165.7
